In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from scipy.stats import permutation_test, f_oneway
import matplotlib.pyplot as plt


In [ ]:
train = pd.read_csv('DataCleaning/train.csv')
chief_complaint = train[['chiefcomplaint', 'stay_length_minutes']]
train = train.drop(columns = ['subject_id', 'hadm_id', 'stay_id', 'race', 'pain', 'intime', 'outtime', 'chiefcomplaint'])

train['race_condensed'] = train['race_condensed'].fillna('Missing')

In [ ]:
# Example data (chief complaints and corresponding treatment times)
data = pd.DataFrame({
    "chief_complaint": ["Chest pain", "Abd pain", "Dyspnea", "SI", "s/p Fall"],
    "treatment_time": [30, 40, 35, 50, 25]  # Treatment times in minutes
})

# Tokenize complaints
data["tokenized_complaint"] = data["chief_complaint"].str.lower().str.split()

# Train a Word2Vec model on all complaints
word2vec_model = Word2Vec(sentences=data["tokenized_complaint"], vector_size=100, window=5, min_count=1, workers=4)

# Custom Transformer for Word2Vec Embeddings
class Word2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model):
        self.model = model
        self.vector_size = model.vector_size

    def fit(self, X, y=None):
        return self  # No fitting needed

    def transform(self, X):
        return np.array([self._get_sentence_embedding(words) for words in X])

    def _get_sentence_embedding(self, words):
        vectors = [self.model.wv[word] for word in words if word in self.model.wv]
        return np.mean(vectors, axis=0) if vectors else np.zeros(self.vector_size)

# Pipeline for processing text features
text_pipeline = FeatureUnion([
    ("tfidf", TfidfVectorizer()),  # TF-IDF feature extraction
    ("word2vec", Word2VecTransformer(model=word2vec_model))  # Word2Vec feature extraction
])

# Full pipeline: Preprocessing + Model Training
pipeline = Pipeline([
    ("features", text_pipeline),
    ("model", RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(data["chief_complaint"], data["treatment_time"], test_size=0.2, random_state=42)

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

print("Predictions:", y_pred)